# Exploring Stack Exchance

While everyone *loves* a fun dataset to explore, good data is expensive. It costs a significant amount of resources to generate, accurately curate, securely store, and provide robust access to. For instance, our *relatively* cold-storage tape archive, [Ranch](https://www.tacc.utexas.edu/systems/ranch), grows at a rate of 8.5PB (~5.3%) per year. Despite the cost, data is often invaluable to both users and administrators.

Everyone that is attending this camp has probaby interacted with at least one Stack Exchange Community:

[**All Sites**](https://stackexchange.com/sites)
- StackOverflow
- Super User
- TeX - LaTeX
- ...and more

An understanding of where knowledge is lacking in your scientific or technological community will allow you to improve your contributions as a whole.

In this section, we will be utilizing our python skills to explore question and answer history from [Stack Exchange](https://stackexchange.com/) that is made available over a RESTful interface. We will be querying and analyzing their **actual** data, and these methods can be extended to a variety of other datasets and websites.

## Objectives

- Use python requests to download data
- Import data into [Pandas](http://pandas.pydata.org/)
- Explore data
  - Inspect and summarize data
  - Group records
  - Select and subset records
  - Visualize selection
  - Join two datasets together

## Dependencies

- [**requests** library](http://docs.python-requests.org/en/master/) *\(Already Installed\)*
- [**pandas** library](http://pandas.pydata.org/) *\(Already Installed\)*

In [ ]:
# Import necessary Libraries
import requests
import pandas as pd
import json
%matplotlib inline

# Stack Exchange Questions

Stack Exchange has a well documented [API](https://api.stackexchange.com/), which contains enpoints for **each site**. While the API allowed for any interaction while authenticated, general information can be retrieved without creating an account.

Beginning with the initial questsions submitted by users, take a look at the

[Questions API](https://api.stackexchange.com/docs/questions)

webapp on the main site, and build a query that you would like to use with Python.

## Goals
- Choose a site (default is StackOverflow)
- Choose Start and/or End Date
- Sort by creation
- Limit the number of questions to 1000 (`pagesize`)

# Transfer to Python

In [ ]:
url = 'https://api.stackexchange.com/2.2/questions'

params = dict(
    site='stackoverflow', # stackoverflow (coding) questions
    pagesize='10',        # Number of questions to return
    fromdate='1500163200',#Get epoch time from website
    order='desc',
    sort='creation'
)

import json
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

print(json.dumps(data, indent=3))

Great! If you kept `pagesize` at 10, you should have a JSON response of 10 questsions. If you decided to crank up your response size, you might have to scroll a bit.

## JSON Structure

This JSON response probably looks familiar if you have ever worked with Python dictionaries in the past. At the most basic level, a JSON is a collection of key, value pairs.

```json
{
    "key1": value1,
    "key2": value2
}
```

Instead of using a numerical index, you refer to each value with the corresponding key. While this makes the data structure human-readable and single-value retrieval and convenient, the nested nature makes makes traditional iteration and indices somewhat difficult.

In [ ]:
# Print first question title
############################
# Pull "items" json
#  > Pull first record
#    > Pull title
print(data['items'][0]['title'])

In [ ]:
# You need to know the exact key names to traverse it
for item in data['items']:
    print("TAGS - %s"%(item['title']))
    print("   [%s]\n"%(", ".join(item['tags'])))

## Explore

- Try pulling out the `answer_count` for each question
- Try pulling out the `view_count` for each question
- Try pulling out the submission date.
- **Extra Credit** - [Convert the epoch time to human readable](https://stackoverflow.com/a/12400584)

# Converting to Pandas

Instead of testing you on your ability to traverse a JSON tree, the goal for today is to explore data using Pandas, so lets convert the JSON to a DataFrame.

In [ ]:
questionsDF = pd.io.json.json_normalize(data['items'])
questionsDF

[`json_normalize`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html) takes a nested JSON and flattens it into a table. In our case, it flattened each return question in the `items` list. Child JSONs like owner, which described the original submitter, now have owner as a prefix in the column name.

### JSON
```
"owner": {
            "reputation": 1,
            "user_id": 6140730,
            "user_type": "registered",
            "profile_image": "https://www.gravatar.com/avatar/efa02138df0bc1f59618c365872caed6?s=128&d=identicon&r=PG&f=1",
            "display_name": "John",
            "link": "https://stackoverflow.com/users/6140730/john"
         }
```
### Table

| Column Name | Value |
|--|--|
| owner.reputation | 1 |
| owner.user_id | 6140730 |
| owner.user_type | registered |
| owner.profile_image | https://www... |
| owner.display_name | John |
| owner.link | https://stackoverflow... |

# Exploring the Data

When we transform the JSON data into a table, using `json_normalize`, the resulting table is actually a [Pandas DataFrame.](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe)

A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, factors and more) in columns. It is similar to a spreadsheet or an SQL table or the data.frame in R. A DataFrame always has an index (0-based). An index refers to the position of an element in the data structure.

You can see the **bold** index column on the left of our example.

## Viewing DataFrame Attributes

Besides having column headers, DataFrames come with some nice attributes and methods to view specific parts of the data.

## Columns

You often need to iterate over the columns of your table, and DataFrames expose those names

In [ ]:
print(questionsDF.columns)

## Shape

You can also see how many rows and columns (rows, columns) are in your DataFrame by accessing the shape attribute.

In [ ]:
print(questionsDF.shape)

## Head

If you have ever used the `head` command on a terminal to view the first N lines of a file, the head function of a DataFrame will look familiar to you.

In [ ]:
questionsDF.head()
#questionsDF.head(2)

## Tail

There is also a tail command for looking at the last N rows of a DataFrame.

In [ ]:
questionsDF.tail()
#questionsDF.tail(2)

# Grouping Records

You may not actually want to use each column in this table, so you can generate summary statistics grouped by subsets or attributes. For example, we could be interested in looked at the `view_count` of each question, and see how many people might also have this problem.

## Column Groups

You can pull out the data for `view_count` using the syntax

```
DF["column name"]
```

We can also describe column numerically using the `describe()` function.

In [ ]:
questionsDF['view_count'].describe()

Besides only using the first 10 questions in my example data, they're all very new, so they have very few views. Lets instead work on the latest 1,000 questions and generate the same description. Because they limit pagesize of the response, we will be pulling the first 100 pages.

In [ ]:
# Latest 1000 questions

# Params pull 100 questsions per query
params = dict(
    site='stackoverflow',
    pagesize='100',
    page='1',
    order='desc',
    sort='creation'
)

nPages = 10 #How many pages you want
data = []

import sys
print("Reading Page:")
for page in map(str, range(1,nPages+1)):
    params['page']=page # Change page number
    if int(page) > 1: sys.stdout.write(", ")
    sys.stdout.write("%s"%(page))
    data += json.loads(requests.get(url=url, params=params).text)['items']

questionsDF = pd.io.json.json_normalize(data)
# Drop the "migrated_from" columns
questionsDF = questionsDF[list(filter(lambda x: 'migrated' not in x, questionsDF.columns))]

questionsDF['view_count'].describe()

Now that we have a larger pool of data, you should check out other statistics that can be generated per column. Feel free to use another numerical column as well.

## Explore

There are a bunch of [built in](https://pandas.pydata.org/pandas-docs/stable/api.html#computations-descriptive-stats) descriptive functions, but these are good to check out.

- describe()
- nuniqe()
- value_counts()

In [ ]:
# How many unique users?
questionsDF['owner.user_id'].nunique()

## Two-Way Groups

If you ever want to summarize by one or more variables, you can use the `groupby` method. In our case, it would be interesting to look at `view_count` statistics of answered and unanswered questions.

In [ ]:
questionsDF.groupby('is_answered')['view_count'].describe()

You can see that while there are fewer answered questions, their view count is almost 100% higher. Neat!

## Explore

Take some time using the `groupby` method to explore other cool trends.

- Owner reputation
- Score

In [ ]:
#questionsDF.groupby('is_answered')['owner.reputation'].describe()

# Selecting and Subsetting Records

You can also select a subset of the data using criteria. For example, we can select all rows that have a `view_count` greater than 5.

In [ ]:
questionsDF[questionsDF.view_count > 5]

## Explore

Experiment with the

- `>`, `<`
- `==`, `!=`
- `>=`, `<=`

operators on numerical data. If you have extra time, look for questions that contain tags that you know. The tags are actually a list, so you can search for tags using the `in` operator.

In [ ]:
questionsDF[ questionsDF.tags.map(lambda x: 'python' in x) ]

# Plotting the Data

While the tables we have been generating are nice, they still contain thousands of rows. A single figure could help visualize the data as a whole. Insead of crafting specific matplotlib calls, Pandas built a universal `plot()` function into the DataFrame object to make it even easier to generate figures. You can plot single columns as histograms

In [ ]:
questionsDF['view_count'].plot(kind='hist')
# Try increasing the resolution with the "bins" parameter
# Try a square root transform of the view count

You can also plot the two way tables we previously made

In [ ]:
questionsDF.groupby('is_answered')['view_count'].plot(kind='hist', legend=True)

## Explore

Try generating a few figures on your own.

# Joining Tables

You can even join two datasets. Lets grab some answers so we can try joining them to their corresponding questions.

In [ ]:
url = 'https://api.stackexchange.com/2.2/answers'
params = dict(
    site='stackoverflow',
    pagesize='100',
    page='1',
    order='desc',
    sort='creation'
)

nPages = 10 #How many pages you want
data = []

import sys
print("Reading Page:")
for page in map(str, range(1,nPages+1)):
    params['page']=page # Change page number
    if int(page) > 1: sys.stdout.write(", ")
    sys.stdout.write("%s"%(page))
    data += json.loads(requests.get(url=url, params=params).text)['items']

answersDF = pd.io.json.json_normalize(data)
answersDF.head()

## Inner Join

The inner of join of S1 and S2
```
S1 - {A1, B1, B2, C1}
S2 - {D1, B1, B2, C1}
```
would equal

```
S1B1 - S2B1
S1B1 - S2B2
S1B2 - S2B1
S1B2 - S2B2
S1C1 - S2C1
```

In [ ]:
merged = pd.merge(left=questionsDF, right=answersDF[['answer_id','question_id']], left_on="question_id", right_on="question_id")
print(merged.shape)
merged.head()
print(questionsDF.columns)

## Left Join

Left joins return all items from the first set, and any items from the second set that overlap with the first. This is useful if we want ALL questions returned, and any questions that also match.

Using the table from the first example, a left join would yield

```
S1A1 - NA
S1B1 - S2B2
S1B2 - S2B1
S1B2 - S2B2
S1C1 - S2C1
```

Notice that whenever there is no match on the right, fields are filled in as NA.

In [ ]:
merged = pd.merge(left=questionsDF, right=answersDF, left_on="question_id", right_on="question_id", how="left")
print(merged.shape)
merged.head()

## Explore

There are also Right and Outer joins to explore. Take a look at [the documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) and see if you can discover anythign fun.

In [ ]:
# Try joining some data